In [1]:
from Example01 import TextToFile
from tempfile import TemporaryDirectory
import os
import shutil

PyTrack creates files and folders for us.
This is an example so we are using a temporary directory and clean it up at th end.

In [2]:
temp_dir = TemporaryDirectory()
shutil.copy('Example01.py', temp_dir.name)
os.chdir(temp_dir.name)

!git init
!dvc init

Initialized empty Git repository in C:/Users/fabia/AppData/Local/Temp/tmpuva0l5v0/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


Content of `Example01.py`
```python
from pytrack import PyTrack


class TextToFile(PyTrack):

    def __init__(self, id_=None, filter_=None):
        super().__init__(id_, filter_)
        self.post_init(id_, filter_)

    def __call__(self, text: str):
        self.parameters = {'text': text}
        self.post_call()

    def run(self):
        self.results = self.parameters

```
This script is a minimal Example that stores a single value `text` as a result


In [3]:
text_to_file = TextToFile()

In [4]:
text_to_file(text="First Test")


In [5]:
!dvc repro


Running stage 'TextToFile_0':
> python -c "from Example01 import TextToFile; TextToFile(id_=0).run()"
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


Content of `dvc.yaml`

```yaml
stages:
  TextToFile_0:
    cmd: python -c "from Example01 import TextToFile; TextToFile(id_=0).run()"
    params:
    - config/params.json:
      - TextToFile.0
    outs:
    - outs\0_TextToFile.json
```
Content of `config\params.json`

```json
{
    "TextToFile": {
        "0": {
            "text": "First Test"
        }
    }
}
```

Now we want to save this version using git.

In [6]:
!git add .
!git commit -m "First Test"

[master (root-commit) 0009b73] First Test
 15 files changed, 558 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvc/plots/confusion.json
 create mode 100644 .dvc/plots/confusion_normalized.json
 create mode 100644 .dvc/plots/default.json
 create mode 100644 .dvc/plots/linear.json
 create mode 100644 .dvc/plots/scatter.json
 create mode 100644 .dvc/plots/smooth.json
 create mode 100644 .dvcignore
 create mode 100644 Example01.py
 create mode 100644 __pycache__/Example01.cpython-39.pyc
 create mode 100644 config/params.json
 create mode 100644 dvc.lock
 create mode 100644 dvc.yaml
 create mode 100644 outs/.gitignore


With everything saved we can create a second Entry save this one with another commit.

In [7]:
text_to_file(text="Second Test")

!dvc repro
!git add .
!git commit -m "Second Test"

ERROR: Stage 'TextToFile_0' already exists in 'dvc.yaml'. Use '--force' to overwrite.



Running stage 'TextToFile_0':
> python -c "from Example01 import TextToFile; TextToFile(id_=0).run()"
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.
[master 9f094e1] Second Test
 2 files changed, 4 insertions(+), 4 deletions(-)


We can use `dvc exp show -A` to look at our commit and also get an overview over the passed parameters.

In [8]:
!dvc exp show -A --sha

+-------------------------------------------+
| Experiment | Created  | TextToFile.0.text |
|------------+----------+-------------------|
| workspace  | -        | Second Test       |
| 9f094e1    | 08:39 PM | Second Test       |
| 0009b73    | 08:39 PM | First Test        |
+-------------------------------------------+


We can now access our saved state via `git checkout` and `dvc checkout`.
For a full explanation on DVC please visit www.dvc.org

In [9]:
!git checkout 0009b73
!dvc checkout
text_to_file.results

Note: switching to '0009b73'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 0009b73 First Test


M       outs\0_TextToFile.json


{'text': 'First Test'}

In [11]:
!git checkout 9f094e1
!dvc checkout
text_to_file.results

HEAD is now at 9f094e1 Second Test


{'text': 'Second Test'}

Because this was a Test, we remove all created files.

In [ ]:
os.chdir('..')
temp_dir.cleanup()